<a href="https://colab.research.google.com/github/JoshmStanley/ggml_models/blob/main/stable_beluga_ggml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Required packages

In [1]:
!pip -q install --upgrade huggingface_hub transformers sentence_transformers accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00


In [2]:
CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"
FORCE_CMAKE=1

In [3]:
!pip -q install --upgrade llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 161.7 MB/s eta 0:00:00


###Model setup

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_id = "jmstanley/stabilityai-StableBeluga2-GGML"
model_filename = "Stable-Beluga-2-ggml.ggmlv3.Q4_K_M.bin"
model_path = hf_hub_download(repo_id=model_id, filename=model_filename)

model = Llama(model_path=model_path,
    #n_gpu_layers=40,
    n_gqa=8,
    n_batch = 256,
    verbose=False,
    n_ctx=2048,
    f16_kv=True
)

### Prompt format and a couple of basic prompts

In [ ]:
#Basic function that formats the prompt in a way that will be useful to the model
def format_prompt(system, examples, instruction, input):
    template = f"### System:\n{system} \n\n### Examples:\n{examples} \n\n### Instruction:\n{instruction} \n\n### Input:\n{input} \n\n### Response:\n"
    return template

prompts = [{'system': 'You are an AI assistant that helps extract concepts from documents related to clinical trials.',
                        'examples': "The overall survival of the enrolled patients from start the combination treatment of Irinotecan, Oxaliplatin, and S1. JSON {'endpoint': 'overall survival', 'endpoint-type': 'Efficacy'}\n \
                                    Example:2 To evaluate the PK profile of GSK30238 on patients with NASH. JSON {endpoint: 'evaluate the PK profile of GSK30238', 'endpoint-type': 'Pharmacokinetics'}\n \
                                    Example:3 To evaluate the safety of regorafenib. JSON {'endpoint': 'evaluate the safety of regorafenib', 'endpoint-type': 'Safety'}",
                        'instruction': "Extract the endpoint from the following statement and return your result with the format {'endpoint': 'endpoint_name', 'endpoint-type': 'endpoint_type'}",
                        'input': "To assess the efficacy and safety of GSK3923 on patients with HER2+ Breast Cancer"},

           {'system': 'You are an AI assistant that helps extract concepts from documents related to clinical trials.',
                        'examples': "The overall survival of the enrolled patients from start the combination treatment of Irinotecan, Oxaliplatin, and S1. {endpoint: 'overall survival', 'endpoint-type': 'Efficacy'} \n \
                                    Example:2 To evaluate the PK profile of GSK30238 on patients with NASH. {endpoint: 'evaluate the PK profile of GSK30238', 'endpoint-type': 'Pharmacokinetics'}\n \
                                    Example:3 To evaluate the safety of regorafenib. {'endpoint': 'evaluate the safety of regorafenib', 'endpoint-type': 'Safety'}",
                        'instruction': "Extract the endpoint and the endpoint name from the following statement. Format the result in a JSON object like {'endpoint': 'endpoint_name', 'endpoint-type': 'endpoint_type'}. If there are multipe endpoints, put them in a list of JSON objects.",
                        'input': "To assess the efficacy and safety of GSK3923 on patients with HER2+ Breast Cancer"}]

#Pass one of our prompts to the prompt format function
prompt = format_prompt(prompts[1]['system'], prompts[1]['examples'], prompts[1]['instruction'], prompts[1]['input'])

In [18]:
#Pass the formatted prompt to the model then print the result.
#Show entire prompt that was passed to the model and the models response after '### Response:'
output = model(prompt, max_tokens=100, stop=["Q:", "\n", "System", "Instruction", "Response"], echo=True)
print(output['choices'][0]['text'])

### System:
You are an AI assistant that helps extract concepts from documents related to clinical trials. 

### Examples:
The overall survival of the enrolled patients from start the combination treatment of Irinotecan, Oxaliplatin, and S1. {endpoint: 'overall survival', 'endpoint-type': 'Efficacy'} 
                                     Example:2 To evaluate the PK profile of GSK30238 on patients with NASH. {endpoint: 'evaluate the PK profile of GSK30238', 'endpoint-type': 'Pharmacokinetics'}
                                     Example:3 To evaluate the safety of regorafenib. {'endpoint': 'evaluate the safety of regorafenib', 'endpoint-type': 'Safety'} 

### Instruction:
Extract the endpoint and the endpoint name from the following statement. Format the result in a JSON object like {'endpoint': 'endpoint_name', 'endpoint-type': 'endpoint_type'}. If there are multipe endpoints, put them in a list of JSON objects. 

### Input:
To assess the efficacy and safety of GSK3923 on patients wi